In [2]:
import os
import argparse
import torch
import random
import numpy as np
from tqdm import tqdm
from torch.utils.data import random_split

from model.crnn import CRNN
from dataset import DatasetImg
from utils.utils import *
from trainer import *

In [3]:
pretrained = 'pretrain/model_50.pth.tar'

In [4]:
device = ( "cuda" if torch.cuda.is_available() else "cpu")
print("---------------------------------------------------")
print(f"Using {device} device")
print("---------------------------------------------------")

dataset = DatasetImg('data/img', 'data/label', 512, 32)

with open(os.path.join('data/char.txt'), 'r', encoding='utf-8') as f:
    alphabet = f.read().rstrip()
# print(alphabet)
converter = strLabelConverter(alphabet, ignore_case=True)

# --------------------- Create Model ---------------------------------
model = CRNN(converter.numClass, 100, 0.1).to(device)
# print(f"Model structure: {model}\n\n")
# for name, param in model.named_parameters():
#     print(f"Layer: {name} | Size: {param.size()} | Values : {param[:2]} \n")

criterion = torch.nn.CTCLoss().to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

epoch = 0
if pretrained:
    checkpoint_path = pretrained
    checkpoint = torch.load(checkpoint_path, map_location=torch.device(device))
    model.load_state_dict(checkpoint['model_state_dict'])
    optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
    epoch = checkpoint['epoch']

---------------------------------------------------
Using cpu device
---------------------------------------------------


In [ ]:
from tester import Tester

tester = Tester()

In [8]:
train_dataloader = torch.utils.data.DataLoader(
                    dataset,
                    batch_size=1,
                    shuffle=True)

In [9]:
imgs, labels = next(iter(train_dataloader))

In [37]:
from utils.utils import *
from tqdm import tqdm
print("Tester.eval...")
imgs, labels = next(iter(train_dataloader))
model.eval()
with torch.no_grad():
    avg_loss = 0
    avg_levenshtein_loss = 0

    imgs = imgs.to(device)

    targets, target_lenghts = converter.encode(labels)
    targets = targets.to('cpu')
    target_lenghts = target_lenghts.to('cpu')

    preds = model(imgs)
    
    b, l, c = preds.shape
    preds_ = preds.permute(1, 0, 2).to('cpu')
    preds_lengths = torch.full(size=(b,), fill_value=l, dtype=torch.long).to('cpu')

    loss = criterion(preds_.log_softmax(2), targets, preds_lengths, target_lenghts) # ctc_loss chỉ dùng với cpu, dùng với gpu phức tạp hơn thì phải

    avg_loss += loss.detach().item()
    _, enc_preds = preds.max(2)
    sim_preds = converter.decode(enc_preds.view(-1), preds_lengths, raw = False)
    # avg_levenshtein_loss += Levenshtein_loss(sim_preds, labels)
      
    raw_preds = converter.decode(enc_preds.view(-1), preds_lengths, raw = True)
    
    print('='*30)
    print(f'raw: {raw_preds}')
    print(f'pred_text: {sim_preds}', )
    print(f'gt: {labels}')

Tester.eval...
raw: ccuóócc  sììnng  ssú   chhã  ccho  bâân  cqq  nnhăăngg  nggáõii  {âât  {{áúúcc tthffnhh crrng  thheeo  mótt  ccgcch ggiããnn ddèè
pred_text: cuóc sìng sú chã cho bân cq nhăng ngáõi {ât {áúc thfnh crng theo mót cgch giãn dè
gt: ('Cuộc sống sẽ chỉ cho bạn có những người đạt được thành công theo một cách giản dị',)
